In [ ]:
import numpy as np
import argparse
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.nn.modules.distance import PairwiseDistance
from datasets.LFWDataset import LFWDataset
from datasets.AIHubDataset import AIHubDataset
from losses.triplet_loss import TripletLoss
from datasets.TripletLossDataset import TripletFaceDataset
from validate_on_LFW import evaluate_lfw
from validate_aihub import validate_aihub
from plot import plot_roc_lfw, plot_accuracy_lfw
from tqdm import tqdm
from model.inceptionresnetv2 import InceptionResnetV2Triplet
from model.mobilenetv2 import MobileNetV2Triplet
from model.resnet import (
    Resnet18Triplet,
    Resnet34Triplet,
    Resnet50Triplet,
    Resnet101Triplet,
    Resnet152Triplet,
)

In [ ]:
def set_model_architecture(model_architecture, pretrained, embedding_dimension):
    if model_architecture == "resnet18":
        model = Resnet18Triplet(
            embedding_dimension=embedding_dimension, pretrained=pretrained
        )
    elif model_architecture == "resnet34":
        model = Resnet34Triplet(
            embedding_dimension=embedding_dimension, pretrained=pretrained
        )
    elif model_architecture == "resnet50":
        model = Resnet50Triplet(
            embedding_dimension=embedding_dimension, pretrained=pretrained
        )
    elif model_architecture == "resnet101":
        model = Resnet101Triplet(
            embedding_dimension=embedding_dimension, pretrained=pretrained
        )
    elif model_architecture == "resnet152":
        model = Resnet152Triplet(
            embedding_dimension=embedding_dimension, pretrained=pretrained
        )
    elif model_architecture == "inceptionresnetv2":
        model = InceptionResnetV2Triplet(
            embedding_dimension=embedding_dimension, pretrained=pretrained
        )
    elif model_architecture == "mobilenetv2":
        model = MobileNetV2Triplet(
            embedding_dimension=embedding_dimension, pretrained=pretrained
        )
    print("Using {} model architecture.".format(model_architecture))

    return model

In [ ]:
def set_model_gpu_mode(model):
    flag_train_gpu = torch.cuda.is_available()
    flag_train_multi_gpu = False

    if flag_train_gpu and torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
        model.cuda()
        flag_train_multi_gpu = True
        print("Using multi-gpu training.")

    elif flag_train_gpu and torch.cuda.device_count() == 1:
        model.cuda()
        print("Using single-gpu training.")

    return model, flag_train_multi_gpu

In [ ]:
def set_optimizer(optimizer, model, learning_rate):
    if optimizer == "sgd":
        optimizer_model = optim.SGD(
            params=model.parameters(),
            lr=learning_rate,
            momentum=0.9,
            dampening=0,
            nesterov=False,
            weight_decay=1e-5,
        )

    elif optimizer == "adagrad":
        optimizer_model = optim.Adagrad(
            params=model.parameters(),
            lr=learning_rate,
            lr_decay=0,
            initial_accumulator_value=0.1,
            eps=1e-10,
            weight_decay=1e-5,
        )

    elif optimizer == "rmsprop":
        optimizer_model = optim.RMSprop(
            params=model.parameters(),
            lr=learning_rate,
            alpha=0.99,
            eps=1e-08,
            momentum=0,
            centered=False,
            weight_decay=1e-5,
        )

    elif optimizer == "adam":
        optimizer_model = optim.Adam(
            params=model.parameters(),
            lr=learning_rate,
            betas=(0.9, 0.999),
            eps=1e-08,
            amsgrad=False,
            weight_decay=1e-5,
        )

    return optimizer_model

In [ ]:
def validate_lfw(model, lfw_dataloader, model_architecture, epoch):
    model.eval()
    with torch.no_grad():
        l2_distance = PairwiseDistance(p=2)
        distances, labels = [], []

        print("Validating on LFW! ...")
        progress_bar = enumerate(tqdm(lfw_dataloader))

        for batch_index, (data_a, data_b, label) in progress_bar:
            data_a = data_a.cuda()
            data_b = data_b.cuda()

            output_a, output_b = model(data_a), model(data_b)
            distance = l2_distance.forward(output_a, output_b)  # Euclidean distance

            distances.append(distance.cpu().detach().numpy())
            labels.append(label.cpu().detach().numpy())

        labels = np.array([sublabel for label in labels for sublabel in label])
        distances = np.array(
            [subdist for distance in distances for subdist in distance]
        )

        (
            true_positive_rate,
            false_positive_rate,
            precision,
            recall,
            accuracy,
            roc_auc,
            best_distances,
            tar,
            far,
        ) = evaluate_lfw(distances=distances, labels=labels, far_target=1e-3)
        # Print statistics and add to log
        print(
            "Accuracy on LFW: {:.4f}+-{:.4f}\tPrecision {:.4f}+-{:.4f}\tRecall {:.4f}+-{:.4f}\t"
            "ROC Area Under Curve: {:.4f}\tBest distance threshold: {:.2f}+-{:.2f}\t"
            "TAR: {:.4f}+-{:.4f} @ FAR: {:.4f}".format(
                np.mean(accuracy),
                np.std(accuracy),
                np.mean(precision),
                np.std(precision),
                np.mean(recall),
                np.std(recall),
                roc_auc,
                np.mean(best_distances),
                np.std(best_distances),
                np.mean(tar),
                np.std(tar),
                np.mean(far),
            )
        )
        with open("logs/lfw_{}_log_triplet.txt".format(model_architecture), "a") as f:
            val_list = [
                epoch,
                np.mean(accuracy),
                np.std(accuracy),
                np.mean(precision),
                np.std(precision),
                np.mean(recall),
                np.std(recall),
                roc_auc,
                np.mean(best_distances),
                np.std(best_distances),
                np.mean(tar),
            ]
            log = "\t".join(str(value) for value in val_list)
            f.writelines(log + "\n")

    try:
        # Plot ROC curve
        plot_roc_lfw(
            false_positive_rate=false_positive_rate,
            true_positive_rate=true_positive_rate,
            figure_name="plots/roc_plots/roc_{}_epoch_{}_triplet.png".format(
                model_architecture, epoch
            ),
        )
        # Plot LFW accuracies plot
        plot_accuracy_lfw(
            log_file="logs/aihub_{}_log_triplet.txt".format(model_architecture),
            epochs=epoch,
            dataset="AIHub",
            figure_name="plots/accuracies_plots/aihub_accuracies_{}_epoch_{}_triplet.png".format(
                model_architecture, epoch
            ),
        )
    except Exception as e:
        print(e)

    return best_distances

In [ ]:
def forward_pass(imgs, model, batch_size):
    imgs = imgs.cuda()
    embeddings = model(imgs)

    # Split the embeddings into Anchor, Positive, and Negative embeddings
    anc_embeddings = embeddings[:batch_size]
    pos_embeddings = embeddings[batch_size : batch_size * 2]
    neg_embeddings = embeddings[batch_size * 2 :]

    return anc_embeddings, pos_embeddings, neg_embeddings, model

In [ ]:
dataroot = "data/face-image/train_aihub_family"
lfw_dataroot = "data/face-image/lfw_224"
aihub_dataroot = "data/face-image/valid_aihub_family"
training_dataset_csv_path = "aihub_train.csv"
epochs = 2  # 150
iterations_per_epoch = 10  # 5000
model_architecture = "resnet34"
pretrained = True  # False
embedding_dimension = 512
num_human_identities_per_batch = 32
batch_size = 100  # 544
lfw_batch_size = 200
resume_path = ""
num_workers = 1
optimizer = "adagrad"
learning_rate = 0.075
margin = 0.2
image_size = 140
use_semihard_negatives = False
training_triplets_path = None

In [ ]:
flag_training_triplets_path = False
start_epoch = 0

if training_triplets_path is not None:
    flag_training_triplets_path = (
        True  # Load triplets file for the first training epoch
    )

In [ ]:
# Define image data pre-processing transforms
#   ToTensor() normalizes pixel values between [0, 1]
#   Normalize(mean=[0.6071, 0.4609, 0.3944], std=[0.2457, 0.2175, 0.2129]) according to the calculated glint360k
#   dataset with tightly-cropped faces dataset RGB channels' mean and std values by
#   calculate_glint360k_rgb_mean_std.py in 'datasets' folder.
aihub_mean = [0.5444, 0.4335, 0.3800]
aihub_std = [0.2672, 0.2295, 0.2156]
data_transforms = transforms.Compose(
    [
        transforms.Resize(size=(image_size, image_size)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=5),
        transforms.ToTensor(),
        transforms.Normalize(mean=aihub_mean, std=aihub_std),
    ]
)

lfw_transforms = transforms.Compose(
    [
        transforms.Resize(size=(image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.6071, 0.4609, 0.3944], std=[0.2457, 0.2175, 0.2129]
        ),
    ]
)

aihub_transforms = transforms.Compose(
    [
        transforms.Resize(size=(image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=aihub_mean, std=aihub_std),
    ]
)

In [ ]:
lfw_dataloader = torch.utils.data.DataLoader(
    dataset=LFWDataset(
        dir=lfw_dataroot, pairs_path="datasets/LFW_pairs.txt", transform=lfw_transforms
    ),
    batch_size=lfw_batch_size,
    num_workers=num_workers,
    shuffle=False,
)

aihub_dataloader = torch.utils.data.DataLoader(
    dataset=AIHubDataset(
        dir=aihub_dataroot,
        pairs_path="data/pairs/valid/pairs_Family.txt",
        transform=aihub_transforms,
    ),
    batch_size=lfw_batch_size,
    num_workers=num_workers,
    shuffle=False,
)

In [ ]:
# Instantiate model
model = set_model_architecture(
    model_architecture=model_architecture,
    pretrained=pretrained,
    embedding_dimension=embedding_dimension,
)

# Load model to GPU or multiple GPUs if available
model, flag_train_multi_gpu = set_model_gpu_mode(model)

# Set optimizer
optimizer_model = set_optimizer(
    optimizer=optimizer, model=model, learning_rate=learning_rate
)

In [ ]:
# Resume from a model checkpoint
if resume_path:
    if os.path.isfile(resume_path):
        print("Loading checkpoint {} ...".format(resume_path))
        checkpoint = torch.load(resume_path)
        start_epoch = checkpoint["epoch"] + 1
        optimizer_model.load_state_dict(checkpoint["optimizer_model_state_dict"])

        # In order to load state dict for optimizers correctly, model has to be loaded to gpu first
        if flag_train_multi_gpu:
            model.module.load_state_dict(checkpoint["model_state_dict"])
        else:
            model.load_state_dict(checkpoint["model_state_dict"])
        print("Checkpoint loaded: start epoch from checkpoint = {}".format(start_epoch))
    else:
        print(
            "WARNING: No checkpoint found at {}!\nTraining from scratch.".format(
                resume_path
            )
        )

In [ ]:
if use_semihard_negatives:
    print("Using Semi-Hard negative triplet selection!")
else:
    print("Using Hard negative triplet selection!")

In [ ]:
start_epoch = start_epoch

print(
    "Training using triplet loss starting for {} epochs:\n".format(epochs - start_epoch)
)

In [ ]:
for epoch in range(start_epoch, epochs):
    num_valid_training_triplets = 0
    l2_distance = PairwiseDistance(p=2)
    _training_triplets_path = None

    if flag_training_triplets_path:
        _training_triplets_path = training_triplets_path
        flag_training_triplets_path = (
            False  # Only load triplets file for the first epoch
        )

    # Re-instantiate training dataloader to generate a triplet list for this training epoch
    train_dataloader = torch.utils.data.DataLoader(
        dataset=TripletFaceDataset(
            root_dir=dataroot,
            training_dataset_csv_path=training_dataset_csv_path,
            num_triplets=iterations_per_epoch * batch_size,
            num_human_identities_per_batch=num_human_identities_per_batch,
            triplet_batch_size=batch_size,
            epoch=epoch,
            training_triplets_path=_training_triplets_path,
            transform=data_t nnnnnnnnnnnnnnnnnnnnransforms,
        ),
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=False,  # Shuffling for triplets with set amount of human identities per batch is not required
    )

    # Training pass
    model.train()
    # progress_bar = enumerate(tqdm(train_dataloader))
    progress_bar = enumerate(train_dataloader)

    for batch_idx, (batch_sample) in progress_bar:

        # Forward pass - compute embeddings
        anc_imgs = batch_sample["anc_img"]
        pos_imgs = batch_sample["pos_img"]
        neg_imgs = batch_sample["neg_img"]

        # Concatenate the input images into one tensor because doing multiple forward passes would create
        #  weird GPU memory allocation behaviours later on during training which would cause GPU Out of Memory
        #  issues
        all_imgs = torch.cat(
            (anc_imgs, pos_imgs, neg_imgs)
        )  # Must be a tuple of Torch Tensors

        anc_embeddings, pos_embeddings, neg_embeddings, model = forward_pass(
            imgs=all_imgs, model=model, batch_size=batch_size
        )

        pos_dists = l2_distance.forward(anc_embeddings, pos_embeddings)
        neg_dists = l2_distance.forward(anc_embeddings, neg_embeddings)

        if use_semihard_negatives:
            # Semi-Hard Negative triplet selection
            #  (negative_distance - positive_distance < margin) AND (positive_distance < negative_distance)
            #   Based on: https://github.com/davidsandberg/facenet/blob/master/src/train_tripletloss.py#L295
            first_condition = (neg_dists - pos_dists < margin).cpu().numpy().flatten()
            second_condition = (pos_dists < neg_dists).cpu().numpy().flatten()
            all = np.logical_and(first_condition, second_condition)
            valid_triplets = np.where(all == 1)
        else:
            # Hard Negative triplet selection
            #  (negative_distance - positive_distance < margin)
            #   Based on: https://github.com/davidsandberg/facenet/blob/master/src/train_tripletloss.py#L296
            all = (neg_dists - pos_dists < margin).cpu().numpy().flatten()
            valid_triplets = np.where(all == 1)

        anc_valid_embeddings = anc_embeddings[valid_triplets]
        pos_valid_embeddings = pos_embeddings[valid_triplets]
        neg_valid_embeddings = neg_embeddings[valid_triplets]

        # Calculate triplet loss
        triplet_loss = TripletLoss(margin=margin).forward(
            anchor=anc_valid_embeddings,
            positive=pos_valid_embeddings,
            negative=neg_valid_embeddings,
        )

        # Calculating number of triplets that met the triplet selection method during the epoch
        num_valid_training_triplets += len(anc_valid_embeddings)

        # Backward pass
        optimizer_model.zero_grad()
        triplet_loss.backward()
        optimizer_model.step()
    
    # Print training statistics for epoch and add to log
    print('Epoch {}:\tNumber of valid training triplets in epoch: {}'.format(
            epoch,
            num_valid_training_triplets
        )
    )

    with open('logs/{}_log_triplet.txt'.format(model_architecture), 'a') as f:
        val_list = [
            epoch,
            num_valid_training_triplets
        ]
        log = '\t'.join(str(value) for value in val_list)
        f.writelines(log + '\n')

    # Evaluation pass on LFW dataset
    is_aihub = True
    validate = validate_aihub if is_aihub else validate_lfw
    loader = aihub_dataloader if is_aihub else lfw_dataloader
    best_distances = validate(
        model=model,
        aihub_dataloader=loader,
        model_architecture=model_architecture,
        epoch=epoch
    )

    # Save model checkpoint
    state = {
        'epoch': epoch,
        'embedding_dimension': embedding_dimension,
        'batch_size_training': batch_size,
        'model_state_dict': model.state_dict(),
        'model_architecture': model_architecture,
        'optimizer_model_state_dict': optimizer_model.state_dict(),
        'best_distance_threshold': np.mean(best_distances)
    }

    # For storing data parallel model's state dictionary without 'module' parameter
    if flag_train_multi_gpu:
        state['model_state_dict'] = model.module.state_dict()

    # Save model checkpoint
    torch.save(state, 'model_training_checkpoints/model_{}_triplet_epoch_{}.pt'.format(
            model_architecture,
            epoch
        )
    )